In [3]:
from ultralytics import YOLO
import cvzone
import cv2

In [4]:
model= YOLO("yolov10n.pt")

C:\Users\tamal\AppData\Roaming\Python\Python312\site-packages\ultralytics\nn\tasks.py:775: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location

In [2]:
import cv2
import torch
from ultralytics import YOLO  # Assuming you have a YOLO class to load the model

# Check if CUDA is available and use GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load the YOLO model and move it to GPU if available
model = YOLO("yolov10n.pt").to(device)

C:\Users\tamal\AppData\Roaming\Python\Python312\site-packages\ultralytics\nn\tasks.py:775: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location

In [ ]:
model= YOLO("yolov10n.pt")
cap=cv2.VideoCapture(0)
while True:
    try:
        ret, image= cap.read()
        results=model(image)
        for info in results:
            parameters= info.boxes
            for box in parameters:
                x1,y1,x2,y2= box.xyxy[0].numpy().astype("int")
                confidence= int(box.conf[0].numpy()*100)
                index_class= int(box.cls[0])
                name_class= results[0].names[index_class]

                cv2.rectangle(image,(x1,y1),(x2,y2),(0,0,255),3)
                # cvzone.putTextRect(image, f"{name_class}",[x1+8, y1-12], thickness=2)
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(image, f"{name_class} : {confidence/100}", (x1 + 8, y1 - 12), font, 0.5, (255, 255, 255), 2)
        cv2.imshow("frame",image)
        cv2.waitKey(1)

    except:break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import numpy as np
url = 'http://192.168.0.102:81/stream'
cap = cv2.VideoCapture(url)

if not cap.isOpened():
    print("Error: Could not open the video stream.")
    exit()
cv2.namedWindow('ESP32-CAM Stream (RGB)', cv2.WINDOW_NORMAL)
cv2.resizeWindow('ESP32-CAM Stream (RGB)', 800, 600)  # Set window size to 1280x720
while True:
    try:
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Failed to grab frame.")
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imshow('ESP32-CAM Stream (RGB)', rgb_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except: 
        cap.release()
        cv2.destroyAllWindows()
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import threading
import cv2
from ultralytics import YOLO
from PyQt5.QtCore import QTimer, QThread, pyqtSignal

class VideoCaptureThread(threading.Thread):
    def __init__(self, thread_id=1, device_id=0):
        threading.Thread.__init__(self)
        self.thread_id = thread_id
        self.device_id = device_id
        self.capture = None
        self.is_running = False
        self.frame = None
        self.lock = threading.Lock()

    def assign_source(self, link_url):
        self.capture = cv2.VideoCapture(link_url)

    def run(self):
        self.is_running = True
        print(f"Thread-{self.thread_id}: Starting video capture...")
        while self.is_running:
            ret, frame = self.capture.read()
            if ret:
                with self.lock:
                    self.frame = frame

    def get_frame(self):
        with self.lock:
            return self.frame

    def stop(self):
        print(f"Thread-{self.thread_id}: Stopping video capture...")
        self.is_running = False
        if self.capture:
            self.capture.release()


class VideoObjectDetect(threading.Thread):
    def __init__(self, thread_id=2, device_id=0):
        threading.Thread.__init__(self)
        self.thread_id = thread_id
        self.device_id = device_id
        self.model = None
        self.is_running = False
        self.input_frame = None
        self.output_frame = None
        self.lock = threading.Lock()

    def load_model(self, model_name):
        self.model = YOLO(model_name)

    def set_input_frame(self, frame):
        with self.lock:
            self.input_frame = frame

    def get_output_frame(self):
        with self.lock:
            return self.output_frame

    def run(self):
        self.is_running = True
        print(f"Thread-{self.thread_id}: Starting object detection...")
        font = cv2.FONT_HERSHEY_SIMPLEX
        while self.is_running:
            if self.input_frame is None:
                continue
            with self.lock:
                frame = self.input_frame.copy()
            results = self.model(frame)
            for info in results:
                parameters = info.boxes
                for box in parameters:
                    x1, y1, x2, y2 = box.xyxy[0].numpy().astype("int")
                    confidence = int(box.conf[0].numpy() * 100)
                    index_class = int(box.cls[0])
                    name_class = results[0].names[index_class]
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    cv2.putText(frame, f"{name_class} : {confidence / 100}", (x1 + 8, y1 - 12), font, 0.5, (255, 255, 255), 2)
            with self.lock:
                self.output_frame = frame

    def stop(self):
        print(f"Thread-{self.thread_id}: Stopping object detection...")
        self.is_running = False


class ControlSOD:
    def __init__(self, access_url, model_path):
        self.video_capture = VideoCaptureThread(thread_id=1, device_id=0)
        self.object_detect = VideoObjectDetect(thread_id=2, device_id=0)
        self.video_capture.assign_source(access_url)
        self.object_detect.load_model(model_path)

    def start(self):
        self.video_capture.start()
        self.object_detect.start()
        # self.display()

    def display(self):
        print("Starting display loop...")
        while True:
            frame = self.video_capture.get_frame()
            if frame is not None:
                self.object_detect.set_input_frame(frame)
            output_frame = self.object_detect.get_output_frame()
            if output_frame is not None:
                cv2.imshow("Object Detection", output_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.stop()
                break

    def stop(self):
        print("Stopping system...")
        if self.object_detect:
            self.object_detect.stop()
            self.object_detect.join()
        if self.video_capture:
            self.video_capture.stop()
            self.video_capture.join()
        cv2.destroyAllWindows()


CSOD = ControlSOD()
CSOD.start(access_url='http://192.168.0.102:81/stream', model_path="./yolov10n.pt")


Thread-1: Starting video capture...
Thread-2: Starting object detection...


C:\Users\tamal\AppData\Roaming\Python\Python312\site-packages\ultralytics\nn\tasks.py:775: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location

In [9]:
CSOD.stop()

Stopping system...
Thread-2: Stopping object detection...
Thread-1: Stopping video capture...


In [6]:
import cv2
import numpy as np
url = 'http://192.168.0.102:81/stream'
cap = cv2.VideoCapture(url)

if not cap.isOpened():
    print("Error: Could not open the video stream.")
    exit()
cv2.namedWindow('ESP32-CAM Stream (RGB)', cv2.WINDOW_NORMAL)
cv2.resizeWindow('ESP32-CAM Stream (RGB)', 800, 600)  # Set window size to 1280x720
while True:
    try:
        ret, image = cap.read()
        results=model(image)
        for info in results:
            parameters= info.boxes
            for box in parameters:
                x1,y1,x2,y2= box.xyxy[0].numpy().astype("int")
                confidence= int(box.conf[0].numpy()*100)
                index_class= int(box.cls[0])
                name_class= results[0].names[index_class]

                cv2.rectangle(image,(x1,y1),(x2,y2),(0,0,255),3)
                # cvzone.putTextRect(image, f"{name_class}",[x1+8, y1-12], thickness=2)
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(image, f"{name_class} : {confidence/100}", (x1 + 8, y1 - 12), font, 0.5, (255, 255, 255), 2)
        if not ret:
            print("Error: Failed to grab frame.")
            break
        rgb_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.imshow('ESP32-CAM Stream (RGB)', rgb_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except: 
        cap.release()
        cv2.destroyAllWindows()
        break
cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 71.5ms
Speed: 1.4ms preprocess, 71.5ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 73.2ms
Speed: 1.0ms preprocess, 73.2ms inference, 0.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 71.6ms
Speed: 1.5ms preprocess, 71.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 83.0ms
Speed: 3.3ms preprocess, 83.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 71.8ms
Speed: 0.0ms preprocess, 71.8ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 66.1ms
Speed: 1.1ms preprocess, 66.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 70.8ms
Speed: 1.5ms preprocess, 70.8ms inference, 0.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 71.3ms
Speed: 1.3ms preprocess, 71.3ms i

In [ ]:
import cv2
import torch
import threading
import numpy as np
from queue import Queue
from ultralytics import YOLO

# Define a queue to hold frames for processing
frame_queue = Queue(maxsize=10)

# Initialize video capture for the ESP32 stream
url = 'http://192.168.0.102:81/stream'
cap = cv2.VideoCapture(url)

if not cap.isOpened():
    print("Error: Could not open the video stream.")
    exit()

# YOLO model initialization
model = YOLO("yolov10n.pt")  # Replace with your model path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define a function to capture frames from the ESP32 stream
def capture_frames():
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to grab frame.")
            break

        # Put the frame into the queue for processing
        if frame_queue.full():
            frame_queue.get()  # Discard the oldest frame if the queue is full

        frame_queue.put(frame)

# Define a function to perform YOLO object detection on frames
def detect_objects():
    while True:
        if not frame_queue.empty():
            frame = frame_queue.get()  # Get the latest frame from the queue

            # Perform object detection
            results = model(frame)
            for info in results:
                parameters = info.boxes
                for box in parameters:
                    x1, y1, x2, y2 = box.xyxy[0].numpy().astype("int")
                    confidence = int(box.conf[0].numpy() * 100)
                    index_class = int(box.cls[0])
                    name_class = results[0].names[index_class]

                    # Draw bounding box and label
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(frame, f"{name_class} : {confidence / 100}", (x1 + 8, y1 - 12), font, 0.5, (255, 255, 255), 2)

            # Display the frame with detections
            cv2.imshow('ESP32-CAM Stream (RGB)', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Create and start threads for capture and detection
capture_thread = threading.Thread(target=capture_frames)
detection_thread = threading.Thread(target=detect_objects)

capture_thread.daemon = True  # Set as daemon so it exits when the main program exits
detection_thread.daemon = True  # Set as daemon so it exits when the main program exits

capture_thread.start()
detection_thread.start()

# Main thread waits for both threads to finish (in this case, they run indefinitely)
capture_thread.join()
detection_thread.join()

# Cleanup
cap.release()
cv2.destroyAllWindows()


C:\Users\tamal\AppData\Roaming\Python\Python312\site-packages\ultralytics\nn\tasks.py:775: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location


0: 480x640 (no detections), 83.4ms
Speed: 3.4ms preprocess, 83.4ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 67.5ms
Speed: 1.7ms preprocess, 67.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 68.9ms
Speed: 0.0ms preprocess, 68.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 68.6ms
Speed: 1.6ms preprocess, 68.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 83.3ms
Speed: 1.8ms preprocess, 83.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 67.0ms
Speed: 1.2ms preprocess, 67.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 64.6ms
Speed: 1.1ms preprocess, 64.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 66.1ms
Speed: 1.7ms preprocess, 66.1ms i

In [ ]:
import cv2
import torch
import threading
import numpy as np
from queue import Queue
import signal
import sys
from ultralytics import YOLO


# Define a queue to hold frames for processing
frame_queue = Queue(maxsize=10)

# Flag to stop threads
stop_flag = False

# Initialize video capture for the ESP32 stream
url = 'http://192.168.0.102:81/stream'
cap = cv2.VideoCapture(url)

if not cap.isOpened():
    print("Error: Could not open the video stream.")
    exit()

# YOLO model initialization
model = YOLO("yolov10n.pt")  # Replace with your model path
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Function to capture frames from the ESP32 stream
def capture_frames():
    global stop_flag
    while not stop_flag:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to grab frame.")
            break

        # Put the frame into the queue for processing
        if frame_queue.full():
            frame_queue.get()  # Discard the oldest frame if the queue is full

        frame_queue.put(frame)

# Function to perform YOLO object detection on frames
def detect_objects():
    global stop_flag
    while not stop_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()  # Get the latest frame from the queue

            # Perform object detection
            results = model(frame)
            for info in results:
                parameters = info.boxes
                for box in parameters:
                    x1, y1, x2, y2 = box.xyxy[0].numpy().astype("int")
                    confidence = int(box.conf[0].numpy() * 100)
                    index_class = int(box.cls[0])
                    name_class = results[0].names[index_class]

                    # Draw bounding box and label
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(frame, f"{name_class} : {confidence / 100}", (x1 + 8, y1 - 12), font, 0.5, (255, 255, 255), 2)

            # Display the frame with detections
            cv2.imshow('ESP32-CAM Stream (RGB)', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Signal handler for graceful exit
def signal_handler(sig, frame):
    global stop_flag
    print('Exiting...')
    stop_flag = True
    cap.release()
    cv2.destroyAllWindows()

# Set up signal handler to handle interrupt (Ctrl+C) gracefully
signal.signal(signal.SIGINT, signal_handler)

# Create and start threads for capture and detection
capture_thread = threading.Thread(target=capture_frames)
detection_thread = threading.Thread(target=detect_objects)

capture_thread.daemon = True  # Set as daemon so it exits when the main program exits
detection_thread.daemon = True  # Set as daemon so it exits when the main program exits

capture_thread.start()
detection_thread.start()

# Main thread waits for both threads to finish (in this case, they run indefinitely)
capture_thread.join()
detection_thread.join()

# Cleanup after threads have finished
cap.release()
cv2.destroyAllWindows()


C:\Users\tamal\AppData\Roaming\Python\Python312\site-packages\ultralytics\nn\tasks.py:775: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location


0: 480x640 (no detections), 144.1ms
Speed: 9.4ms preprocess, 144.1ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 75.0ms
Speed: 1.1ms preprocess, 75.0ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 keyboards, 64.1ms
Speed: 2.5ms preprocess, 64.1ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 keyboard, 69.2ms
Speed: 1.0ms preprocess, 69.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 72.9ms
Speed: 2.3ms preprocess, 72.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 keyboards, 78.9ms
Speed: 0.0ms preprocess, 78.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 keyboard, 72.9ms
Speed: 0.0ms preprocess, 72.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 keyboard, 78.8ms
Speed: 0.0ms preprocess, 78.8ms inference, 0.0ms postp

In [19]:
# model = YOLO("yolov10n.pt")
# cap = cv2.VideoCapture(0)

# while True:
#     try:
#         ret, image = cap.read()
#         results = model(image)
#         for info in results:
#             parameters = info.boxes
#             for box in parameters:
#                 x1, y1, x2, y2 = box.xyxy[0].numpy().astype("int")
#                 confidence = int(box.conf[0].numpy() * 100)  # Confidence as percentage
#                 index_class = int(box.cls[0])
#                 name_class = results[0].names[index_class]

#                 # Draw bounding box
#                 cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)

#                 # Prepare text with name and confidence
#                 label = f"{name_class} {confidence}%"

#                 # Define font and position for text
#                 font = cv2.FONT_HERSHEY_SIMPLEX
#                 cv2.putText(image, label, (x1 + 8, y1 - 12), font, 0.5, (255, 255, 255), 2)

#         # Display the frame
#         cv2.imshow("frame", image)
        
#         # Exit condition (press 'q' to quit)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     except:
#         break

# cap.release()
# cv2.destroyAllWindows()
